# Solvent Accessible Surface Area for Chromosome Structure

## Project Overview

This project develops an algorithm for calculating the Solvent Accessible Surface Area (SASA) of chromosomes. The algorithm generates a surface around the chromosome, representing the interface where genetic material interacts with intracellular molecules such as proteins, transcription factors, and other DNA-altering compounds.

### Objectives
- Identify accessible regions of the genome exposed to external influences
- Study how 3D genome organization contributes to cancer susceptibility
- Provide insights for gene editing technologies

### Data Source
The input data is a PDB file for chromosome structure from **GSDB** (Genome Structure DataBase).


## 1. Import Libraries and Setup


In [ ]:
# Install required packages
%pip install -q MDTraj

# Import libraries
import mdtraj as md
import pandas as pd
import numpy as np
import scipy.ndimage as nd
import math
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.io as pio
from typing import Tuple, List
import warnings
warnings.filterwarnings('ignore')


## 2. Configuration Parameters

The following hyperparameters control the surface generation algorithm:


In [ ]:
class SASAConfig:
    """Configuration class for SASA calculation parameters."""
    
    def __init__(self):
        self.overlapping_coefficient = 1.45  # oc: overlapping coefficient
        self.sphere_density = 15              # d: density of points in sphere per axis
        self.bins_per_axis = 50               # b: number of bins per axis in the box
        self.padding_factor = 0.1             # padding around the structure
        
    def __repr__(self):
        return f"SASAConfig(oc={self.overlapping_coefficient}, d={self.sphere_density}, b={self.bins_per_axis})"

# Initialize configuration
config = SASAConfig()
print(f"Configuration: {config}")


## 3. Data Loading and Preprocessing


In [ ]:
class ChromosomeLoader:
    """Class for loading and preprocessing chromosome data."""
    
    def __init__(self, url: str):
        self.url = url
        self.coords = None
        self.coords_df = None
        
    def load_data(self) -> np.ndarray:
        """Load chromosome structure from PDB file."""
        print(f"Loading chromosome data from: {self.url}")
        chr_str = md.load_pdb(self.url)
        self.coords = chr_str.xyz.reshape(-1, 3)
        self.coords_df = pd.DataFrame(self.coords, columns=['x', 'y', 'z'])
        print(f"Loaded {len(self.coords)} coordinate points")
        return self.coords
    
    def get_coords(self) -> np.ndarray:
        """Get coordinate array."""
        if self.coords is None:
            raise ValueError("Data not loaded. Call load_data() first.")
        return self.coords
    
    def get_coords_df(self) -> pd.DataFrame:
        """Get coordinate DataFrame."""
        if self.coords_df is None:
            raise ValueError("Data not loaded. Call load_data() first.")
        return self.coords_df

# Load chromosome data
url = 'https://calla.rnet.missouri.edu/genome3d/GSDB/Database/AX9716PF/GSE105544_ENCFF010WBP/VC/LorDG/chr1.pdb'
loader = ChromosomeLoader(url)
coords = loader.load_data()

# Display first few coordinates
print("\nFirst 10 coordinates:")
print(loader.get_coords_df().head(10))


## 4. Chromosome Visualization


In [ ]:
class ChromosomeVisualizer:
    """Class for visualizing chromosome structures."""
    
    @staticmethod
    def plot_chromosome_structure(coords: np.ndarray, title: str = "3D Chromosome Structure") -> go.Figure:
        """Create interactive 3D line plot of chromosome structure."""
        line_dict = dict(width=5, color=coords[:, 2], colorscale='Viridis')
        title_dict = dict(text=title, font=dict(size=24), x=0.5)
        
        data = [go.Scatter3d(
            x=coords[:, 0], 
            y=coords[:, 1], 
            z=coords[:, 2], 
            mode='lines', 
            line=line_dict,
            name='Chromosome'
        )]
        
        fig = go.Figure(data)
        fig.update_layout(
            width=800, 
            height=800, 
            title=title_dict,
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z'
            )
        )
        return fig

# Create chromosome visualization
visualizer = ChromosomeVisualizer()
fig_chr = visualizer.plot_chromosome_structure(coords)
fig_chr.show()


## 5. Distance Calculation and Sphere Generation


In [ ]:
class DistanceCalculator:
    """Class for calculating distances between consecutive chromosome points."""
    
    def __init__(self, coords: np.ndarray):
        self.coords = coords
        self.distances = self._calculate_distances()
    
    def _calculate_distances(self) -> List[float]:
        """Calculate consecutive Euclidean distances."""
        distances = []
        for i in range(len(self.coords) - 1):
            dist = math.dist(self.coords[i, :], self.coords[i + 1, :])
            distances.append(dist)
        return distances
    
    def get_average_distance(self) -> float:
        """Get average distance between consecutive points."""
        return np.mean(self.distances)
    
    def get_max_distance(self) -> float:
        """Get maximum distance between consecutive points."""
        return max(self.distances)
    
    def get_distance_stats(self) -> dict:
        """Get comprehensive distance statistics."""
        return {
            'mean': self.get_average_distance(),
            'max': self.get_max_distance(),
            'min': min(self.distances),
            'std': np.std(self.distances),
            'count': len(self.distances)
        }

class SphereGenerator:
    """Class for generating spheres around chromosome points."""
    
    def __init__(self, config: SASAConfig):
        self.config = config
    
    def create_sphere(self, x: float, y: float, z: float, radius: float) -> np.ndarray:
        """Create a sphere of points around a given coordinate."""
        d = self.config.sphere_density
        
        # Create spherical coordinates
        rad = np.linspace(0, radius, int(d/2))
        phi = np.linspace(0, np.pi, d)
        theta = np.linspace(0, 2*np.pi, d, endpoint=False)
        
        theta, phi, rad = np.meshgrid(theta, phi, rad, indexing='ij')
        phi = phi.flatten()
        theta = theta.flatten()
        rad = rad.flatten()
        
        # Convert to Cartesian coordinates
        dx = rad * np.sin(phi) * np.cos(theta) + x
        dy = rad * np.sin(phi) * np.sin(theta) + y
        dz = rad * np.cos(phi) + z
        
        return np.array([dx, dy, dz])
    
    def generate_all_spheres(self, coords: np.ndarray, radius: float) -> np.ndarray:
        """Generate spheres around all chromosome coordinates."""
        all_spheres = np.empty((3, 0))
        
        print(f"Generating spheres around {len(coords)} points...")
        for point in tqdm(coords, desc="Generating spheres"):
            sphere = self.create_sphere(point[0], point[1], point[2], radius)
            all_spheres = np.hstack([all_spheres, sphere])
        
        return all_spheres.T

# Calculate distances and determine sphere radius
distance_calc = DistanceCalculator(coords)
distance_stats = distance_calc.get_distance_stats()

print("Distance Statistics:")
for key, value in distance_stats.items():
    print(f"  {key}: {value:.4f}")

# Calculate sphere radius
sphere_radius = config.overlapping_coefficient * distance_calc.get_max_distance()
print(f"\nSphere radius: {sphere_radius:.4f}")

# Generate spheres
sphere_gen = SphereGenerator(config)
all_spheres = sphere_gen.generate_all_spheres(coords, sphere_radius)
print(f"Generated {len(all_spheres)} sphere points")

# Scale coordinates to positive values
scaling_factor = abs(np.min(all_spheres) * 1.25)
all_spheres += scaling_factor
coords_scaled = coords + scaling_factor
print(f"Applied scaling factor: {scaling_factor:.4f}")


## 6. 3D Binning and Surface Detection


In [ ]:
class SurfaceDetector:
    """Class for detecting surface points using 3D binning."""
    
    def __init__(self, config: SASAConfig):
        self.config = config
        self.bin_size = None
        self.box = None
        self.surface_coords = None
    
    def create_3d_box(self, all_spheres: np.ndarray) -> np.ndarray:
        """Create 3D box and bin the sphere points."""
        # Calculate box dimensions
        x_lim = [np.min(all_spheres[:, 0]), np.max(all_spheres[:, 0])]
        y_lim = [np.min(all_spheres[:, 1]), np.max(all_spheres[:, 1])]
        z_lim = [np.min(all_spheres[:, 2]), np.max(all_spheres[:, 2])]
        
        edge_size = max(
            x_lim[1] - x_lim[0],
            y_lim[1] - y_lim[0],
            z_lim[1] - z_lim[0]
        )
        
        # Add padding
        padding = edge_size * self.config.padding_factor
        edge_size += padding
        
        # Calculate bin size
        self.bin_size = edge_size / self.config.bins_per_axis
        
        # Initialize 3D box
        b = self.config.bins_per_axis
        self.box = np.zeros((b, b, b), dtype='float')
        
        # Fill box with sphere points
        print(f"Filling 3D box ({b}x{b}x{b}) with {len(all_spheres)} points...")
        for point in tqdm(all_spheres, desc="Binning points"):
            x, y, z = point
            x_idx = int(x // self.bin_size)
            y_idx = int(y // self.bin_size)
            z_idx = int(z // self.bin_size)
            
            if (0 <= x_idx < b) and (0 <= y_idx < b) and (0 <= z_idx < b):
                self.box[x_idx, y_idx, z_idx] = 1
        
        return self.box
    
    def detect_surface(self) -> np.ndarray:
        """Detect surface bins using convolution."""
        if self.box is None:
            raise ValueError("Box not created. Call create_3d_box() first.")
        
        # Create convolution kernel
        kernel = np.ones((3, 3, 3), dtype='int')
        kernel[1, 1, 1] = 0  # Center point excluded
        
        # Count neighbors
        neighbor_count = nd.convolve(self.box, kernel, mode='constant', cval=0)
        
        # Surface bins are those with points but fewer than 26 neighbors
        surface_bins = (self.box == 1) & (neighbor_count < 26)
        
        # Get surface coordinates
        self.surface_coords = np.argwhere(surface_bins == 1)
        
        print(f"Detected {len(self.surface_coords)} surface points")
        return self.surface_coords
    
    def get_surface_coords_scaled(self) -> np.ndarray:
        """Get surface coordinates scaled back to original space."""
        if self.surface_coords is None:
            raise ValueError("Surface not detected. Call detect_surface() first.")
        
        return self.surface_coords * self.bin_size

# Detect surface
surface_detector = SurfaceDetector(config)
box = surface_detector.create_3d_box(all_spheres)
surface_coords = surface_detector.detect_surface()
surface_coords_scaled = surface_detector.get_surface_coords_scaled()

print(f"\nBox dimensions: {box.shape}")
print(f"Bin size: {surface_detector.bin_size:.4f}")
print(f"Total occupied bins: {np.sum(box):.0f}")
print(f"Surface bins: {len(surface_coords)}")


## 7. Final Visualization and Results


In [ ]:
class SASAVisualizer:
    """Class for visualizing SASA results."""
    
    @staticmethod
    def plot_sasa_model(coords: np.ndarray, surface_coords: np.ndarray, 
                       title: str = "SASA for Chromosome") -> go.Figure:
        """Create combined visualization of chromosome and surface."""
        # Surface points
        surface_trace = go.Scatter3d(
            x=surface_coords[:, 0],
            y=surface_coords[:, 1],
            z=surface_coords[:, 2],
            mode='markers',
            marker=dict(size=5, opacity=0.1, color='navy'),
            name='Surface'
        )
        
        # Chromosome trace
        chromosome_trace = go.Scatter3d(
            x=coords[:, 0],
            y=coords[:, 1],
            z=coords[:, 2],
            mode='lines',
            line=dict(width=5, color='red'),
            name='Chromosome'
        )
        
        fig = go.Figure(data=[surface_trace, chromosome_trace])
        fig.update_layout(
            height=800,
            width=800,
            title=dict(text=title, x=0.5, font=dict(size=24)),
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z'
            )
        )
        return fig

# Create final visualization
sasa_viz = SASAVisualizer()
fig_sasa = sasa_viz.plot_sasa_model(coords_scaled, surface_coords_scaled)
fig_sasa.show()

# Save results (optional)
# fig_sasa.write_html("sasa_model.html")
# pio.write_image(fig_sasa, "sasa_model.png", format="png", width=800, height=800, scale=1)


## 8. Results Summary


In [ ]:
def print_results_summary():
    """Print comprehensive results summary."""
    print("=" * 60)
    print("SASA CALCULATION RESULTS SUMMARY")
    print("=" * 60)
    
    print(f"\nConfiguration:")
    print(f"  Overlapping Coefficient: {config.overlapping_coefficient}")
    print(f"  Sphere Density: {config.sphere_density}")
    print(f"  Bins per Axis: {config.bins_per_axis}")
    
    print(f"\nData:")
    print(f"  Chromosome Points: {len(coords)}")
    print(f"  Sphere Points Generated: {len(all_spheres)}")
    print(f"  Scaling Factor Applied: {scaling_factor:.4f}")
    
    print(f"\nDistance Statistics:")
    for key, value in distance_stats.items():
        print(f"  {key.capitalize()}: {value:.4f}")
    
    print(f"\nSurface Detection:")
    print(f"  Box Dimensions: {box.shape}")
    print(f"  Bin Size: {surface_detector.bin_size:.4f}")
    print(f"  Occupied Bins: {np.sum(box):.0f}")
    print(f"  Surface Points: {len(surface_coords)}")
    print(f"  Surface Coverage: {len(surface_coords)/np.sum(box)*100:.2f}%")
    
    print("\n" + "=" * 60)

print_results_summary()


## Future Enhancements

### Planned Improvements:
1. **Dashboard Development**: Create an interactive dashboard with chromosome structure and various plots for calculated physical attributes
2. **Class Refactoring**: Further organize code into classes and define functions for all processes
3. **Physical Quantification**: Use scipy and other scientific packages to quantify various physical attributes
4. **Functional Integration**: Map gene expression, histone modifications, and protein binding sites onto the genomic structure
5. **Cancer Analysis**: Extend framework to study changes in genomic architecture of cancerous cells

### Potential Applications:
- Identify mutation-prone regions in the genome
- Study 3D genome organization in disease states
- Guide gene editing strategies
- Understand structural-functional relationships in genomics
